In [80]:
import numpy as np
import scs
import scipy

In [81]:
#############################################
#      Generate random cone problems        #
#############################################
    
def gen_feasible(m, n, p_scale = 0.1):
    P = p_scale * scipy.sparse.eye(n, format="csc")   
    A = scipy.sparse.random(m, n, density=0.05, format="csc")
    c = np.random.randn(n)
    b = np.random.randn(m)

    P = np.round(P, 1)
    A.data = np.round(A.data, 1)
    b = np.round(b, 1)
    c = np.round(c, 1)

    return (P, A, b, c)

In [82]:
seed = 1234

In [83]:
np.random.seed(seed)
cone = dict(z=1, l=2, s=[3, 4], cs=[5, 4])
m = int(cone['z'] + cone['l'] + sum([j * (j+1) / 2 for j in cone['s']]) + sum([j * j for j in cone['cs']]))
n = m
(P, A, b, c) = gen_feasible(m, n)
probdata = dict(P=P, A=A, b=b, c=c)
sol = scs.solve(probdata, cone, eps_abs=1e-12, eps_rel=1e-12)

------------------------------------------------------------------
	       SCS v3.2.10 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 60, constraints m: 60
cones: 	  z: primal zero / dual free vars: 1
	  l: linear vars: 2
	  s: psd vars: 16, ssize: 2
	  cs: complex psd vars: 41, cssize: 2
settings: eps_abs: 1.0e-12, eps_rel: 1.0e-12, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 180, nnz(P): 60
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.72e+01  2.00e+00  4.38e+02 -3.85e+02  1.00e-01  3.49e-03 
   150| 2.04e-12  4.60e-14  2.10e-12

In [84]:
import numpy as np
import scipy.sparse as sp

def print_c_array(name, data, dtype="scs_float", width=80):
    """
    Helper to format a python iterable into a C array string.
    """
    # Convert data to a list of strings
    if "float" in dtype:
        # Use high precision formatting for floats
        str_data = ["{:.16g}".format(x) for x in data]
    else:
        str_data = [str(int(x)) for x in data]

    output = f"{dtype} {name}[] = {{"
    current_line_len = len(output)
    
    parts = []
    
    for i, val in enumerate(str_data):
        # Determine if we need a comma
        suffix = ", " if i < len(str_data) - 1 else ""
        entry = val + suffix
        
        # Simple line wrapping logic
        if current_line_len + len(entry) > width:
            parts.append("\n    ") # Indent next line
            current_line_len = 4
            
        parts.append(entry)
        current_line_len += len(entry)

    output += "".join(parts) + "};"
    print(output)
    print() # Print an empty line for spacing

def generate_scs_data(P, A, b, c):
    """
    Takes python data and prints C-compatible arrays.
    """
    # Ensure matrices are in CSC format (just in case)
    if not sp.isspmatrix_csc(P):
        P = P.tocsc()
    if not sp.isspmatrix_csc(A):
        A = A.tocsc()

    # Flatten dense vectors if they are 2D arrays
    b = np.ravel(b)
    c = np.ravel(c)

    print("// --- Copy the following into your C file --- \n")

    # --- Process Matrix P ---
    # Px (values), Pi (row indices), Pp (column pointers)
    print_c_array("Px", P.data, "scs_float")
    print_c_array("Pi", P.indices, "scs_int")
    print_c_array("Pp", P.indptr, "scs_int")

    # --- Process Matrix A ---
    # Ax (values), Ai (row indices), Ap (column pointers)
    print_c_array("Ax", A.data, "scs_float")
    print_c_array("Ai", A.indices, "scs_int")
    print_c_array("Ap", A.indptr, "scs_int")

    # --- Process Vectors b and c ---
    print_c_array("b", b, "scs_float")
    print_c_array("c", c, "scs_float")


generate_scs_data(P, A, b, c)

// --- Copy the following into your C file --- 

scs_float Px[] = {0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 
    0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 
    0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 
    0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 
    0.1, 0.1, 0.1};

scs_int Pi[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 
    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 
    37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 
    56, 57, 58, 59};

scs_int Pp[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 
    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 
    37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 
    56, 57, 58, 59, 60};

scs_float Ax[] = {0.8, 0.8, 0.5, 0.4, 0.6, 0.1, 0.4, 0.5, 1, 0.6, 0.4, 0.8, 
    

In [85]:
print(f'{m=}')
print(f'{n=}')

m=60
n=60
